In [1]:
import sys
sys.argv.append('--JOB_NAME')
sys.argv.append('housing-repairs-emergency-lighting-service-cleaning')

sys.argv.append('--source_catalog_database')
sys.argv.append('housing-repairs-raw-zone')

sys.argv.append('--source_catalog_table')
sys.argv.append('housing_repairs_emergency_lighting_servicing')

sys.argv.append('--cleaned_repairs_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-refined-zone/housing-repairs/repairs-electrical-mechanical-fire/emergency-lighting-servicing/cleaned')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pyspark.sql.functions as F
from awsglue.dynamicframe import DynamicFrame
import re

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from helpers import get_glue_env_var, get_latest_partitions, PARTITION_KEYS
from repairs_cleaning_helpers import udf_map_repair_priority, clean_column_names

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'helpers'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'helpers'



In [4]:
import sys
import datetime
import boto3
from awsglue.utils import getResolvedOptions
from pyspark.sql import functions as f

PARTITION_KEYS = ['import_year', 'import_month', 'import_day', 'import_date']

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

def get_secret(logger, secret_name, region_name):
    session = boto3.session.Session()

    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )

    if 'SecretString' in get_secret_value_response:
        return get_secret_value_response['SecretString']
    else:
        return get_secret_value_response['SecretBinary'].decode('ascii')

def add_timestamp_column(data_frame):
    now = datetime.datetime.now()
    return data_frame.withColumn('import_timestamp', f.lit(str(now.timestamp())))

def add_import_time_columns(data_frame):
    now = datetime.datetime.now()
    importYear = str(now.year)
    importMonth = str(now.month).zfill(2)
    importDay = str(now.day).zfill(2)
    importDate = importYear + importMonth + importDay

    data_frame = data_frame.withColumn(
        'import_datetime', f.current_timestamp())
    data_frame = data_frame.withColumn(
        'import_timestamp', f.lit(str(now.timestamp())))
    data_frame = data_frame.withColumn('import_year', f.lit(importYear))
    data_frame = data_frame.withColumn('import_month', f.lit(importMonth))
    data_frame = data_frame.withColumn('import_day', f.lit(importDay))
    data_frame = data_frame.withColumn('import_date', f.lit(importDate))
    return data_frame

def convert_pandas_df_to_spark_dynamic_df(sql_context, panadas_df):
    # Convert to SparkDynamicDataFrame
    spark_df = sql_context.createDataFrame(panadas_df)
    spark_df = spark_df.coalesce(1)
    spark_df = add_import_time_columns(spark_df)

    return spark_df

def get_s3_subfolders(s3_client, bucket_name, prefix):
    there_are_more_objects_in_the_bucket_to_fetch = True
    folders = []
    continuation_token = {}
    while there_are_more_objects_in_the_bucket_to_fetch:
        list_objects_response = s3_client.list_objects_v2(
            Bucket=bucket_name,
            Delimiter='/',
            Prefix=prefix,
            **continuation_token
        )

        folders.extend(x['Prefix']
                       for x in list_objects_response.get('CommonPrefixes', []))
        there_are_more_objects_in_the_bucket_to_fetch = list_objects_response['IsTruncated']
        continuation_token['ContinuationToken'] = list_objects_response.get(
            'NextContinuationToken')

    return set(folders)

def get_latest_partitions(dfa):
    dfa = dfa.where(f.col('import_year') == dfa.select(
        f.max('import_year')).first()[0])
    dfa = dfa.where(f.col('import_month') == dfa.select(
        f.max('import_month')).first()[0])
    dfa = dfa.where(f.col('import_day') == dfa.select(
        f.max('import_day')).first()[0])
    return dfa

import pyspark.sql.functions as F
from pyspark.sql.types import StringType
import re


def map_repair_priority(code):
    if code == 'Immediate':
        return 1
    elif code == 'Emergency':
        return 2
    elif code == 'Urgent':
        return 3
    elif code == 'Normal':
        return 4
    else:
        return None


# # convert to a UDF Function by passing in the function and the return type of function (string in this case)
udf_map_repair_priority = F.udf(map_repair_priority, StringType())


def clean_column_names(df):
    # remove full stops from column names
    df = df.select([F.col("`{0}`".format(c)).alias(
        c.replace('.', '')) for c in df.columns])
    # remove trialing underscores
    df = df.select([F.col(col).alias(re.sub("_$", "", col))
                   for col in df.columns])
    # lowercase and remove double underscores
    df2 = df.select([F.col(col).alias(
        re.sub("[^0-9a-zA-Z$]+", "_", col.lower())) for col in df.columns])
    return df2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
source_catalog_database = get_glue_env_var('source_catalog_database', '')
source_catalog_table = get_glue_env_var('source_catalog_table', '')
cleaned_repairs_s3_bucket_target = get_glue_env_var('cleaned_repairs_s3_bucket_target', '')

args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
source_data = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table,
)

df = source_data.toDF()
df = get_latest_partitions(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df.printSchema()
df3.show(n=50, truncate=100, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Address: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- temp_order_number: string (nullable = true)
 |-- priority_code: string (nullable = true)
 |-- new_uhw_number: string (nullable = true)
 |-- Cost: string (nullable = true)
 |-- status_of_completed__y/n_: string (nullable = true)
 |-- requested_by: string (nullable = true)
 |-- import_datetime: timestamp (nullable = true)
 |-- import_timestamp: string (nullable = true)
 |-- import_year: string (nullable = true)
 |-- import_month: string (nullable = true)
 |-- import_day: string (nullable = true)
 |-- import_date: string (nullable = true)

-RECORD 0--------------------------------------------------------------
 datetime_raised           | 2021-01-25 00:00:00                       
 operative                 | Kimson Alexander                          
 property_address          | 41-52 St Leonards court                   
 description_of_work       | Emergency Li

In [8]:
df2 = clean_column_names(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df2.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- address: string (nullable = true)
 |-- description: string (nullable = true)
 |-- date: string (nullable = true)
 |-- temp_order_number: string (nullable = true)
 |-- priority_code: string (nullable = true)
 |-- new_uhw_number: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- status_of_completed_y_n: string (nullable = true)
 |-- requested_by: string (nullable = true)
 |-- import_datetime: timestamp (nullable = true)
 |-- import_timestamp: string (nullable = true)
 |-- import_year: string (nullable = true)
 |-- import_month: string (nullable = true)
 |-- import_day: string (nullable = true)
 |-- import_date: string (nullable = true)

In [25]:
df3 = df2

# Remove any instances of nan
df3 = df3.replace('nan', None)

# rename column names to reflect harmonised column names
df3 = df3.withColumnRenamed('requested_by', 'operative') \
    .withColumnRenamed('address', 'property_address') \
    .withColumnRenamed('description', 'description_of_work') \
    .withColumnRenamed('priority_code', 'work_priority_description') \
    .withColumnRenamed('temp_order_number', 'temp_order_number_full') \
    .withColumnRenamed('cost', 'order_value')

df3 = df3.withColumn('date', F.to_timestamp('date', "yyyy-MM-dd")).withColumnRenamed('date', 'datetime_raised')
df3 = df3.withColumn('data_source', F.lit('ElecMechFire - Emergency Lighting Service'))
df3 = df3.withColumn('work_priority_priority_code', udf_map_repair_priority('work_priority_description'))
df3 = df3.withColumn('status_of_completed_y_n', F.when(df3['status_of_completed_y_n']=='Y', 'Completed').otherwise(''))\
    .withColumnRenamed('status_of_completed_y_n', 'order_status')

# only keep relevant columns
df3 = df3[[
    'datetime_raised',
    'operative',
    'property_address',
    'description_of_work',
    'work_priority_description',
    'temp_order_number_full',
    'order_value',
    'order_status',
    'data_source',
    'import_datetime',
    'import_timestamp',
    'import_year',
    'import_month',
    'import_day',
    'import_date'
]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df3.printSchema()
df3.show(n=50, truncate=100, vertical=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- datetime_raised: timestamp (nullable = true)
 |-- operative: string (nullable = true)
 |-- property_address: string (nullable = true)
 |-- description_of_work: string (nullable = true)
 |-- work_priority_description: string (nullable = true)
 |-- temp_order_number_full: string (nullable = true)
 |-- order_value: string (nullable = true)
 |-- order_status: string (nullable = false)
 |-- data_source: string (nullable = false)
 |-- import_datetime: timestamp (nullable = true)
 |-- import_timestamp: string (nullable = true)
 |-- import_year: string (nullable = true)
 |-- import_month: string (nullable = true)
 |-- import_day: string (nullable = true)
 |-- import_date: string (nullable = true)

-RECORD 0--------------------------------------------------------------
 datetime_raised           | 2021-01-25 00:00:00                       
 operative                 | Kimson Alexander                          
 property_address          | 41-52 St Leonards court                   
 de

In [ ]:
cleanedDataframe = DynamicFrame.fromDF(df3, glueContext, "cleanedDataframe")
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=cleanedDataframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": cleaned_repairs_s3_bucket_target,"partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")
job.commit()